In [1]:
from common.common_imports import *
from sqlalchemy import create_engine, Table, Index, MetaData, select, distinct, exists
from sqlalchemy.sql import and_, desc
from data.time_encoder import TIME_ENCODER
from data.h5py import H5PY_CONTROLLER
from data.history import HISTORY_DATA
from data.info import INFO_DATA


In [2]:
a = torch.empty(7, dtype=int)
b = torch.full_like(a, 4)
c = torch.asarray([1,1,2,3,4,5,6])
a = torch.min(b,c)
d = torch.full((7,10), True)
e = torch.arange(10).unsqueeze(0).expand(7, -1)
f = torch.empty((7,3))
for i in range(7):
    d[i,:a[i]] = False
    f[i] = e[i, a[i]-1:a[i]-1+3]
# exp = [0,1,2,3,4,4,4]


print(a)
print(d)
print(f)
# history_data = HISTORY_DATA()
# print(
#     history_data.ticker_stoi("AAPL")
# )
# print(
#     history_data.ticker_itos(18)
# )

tensor([1, 1, 2, 3, 4, 4, 4])
tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True]])
tensor([[0., 1., 2.],
        [0., 1., 2.],
        [1., 2., 3.],
        [2., 3., 4.],
        [3., 4., 5.],
        [3., 4., 5.],
        [3., 4., 5.]])


In [3]:
info_data = INFO_DATA()
print(
    info_data.info_tuple_from_index(18)
)
info = info_data.info_array_from_index(18)
print(info)
# info_padded = np.pad(info, (0, history.shape[1]-info.shape[0]), mode='constant', constant_values=np.NINF)

AAPL
('United States', 'Technology', 'Computer Manufacturing')
AAPL
('United States', 'Technology', 'Computer Manufacturing')
[56.  9. 34.]


In [4]:
class DB:
    def __init__(self) -> None:
        self.engine = create_engine('sqlite:////home/kanesweet/kanesweet/stonks.0.1/stonks.db')
        self.connection = self.engine.connect()
        self.metadata = MetaData(bind=self.engine)
        self.history_table = Table('history', self.metadata, autoload=True, autoload_with=self.engine)
        self.attached = False

db = DB()

In [11]:
h5py = H5PY_CONTROLLER()
tickers_idx = pd.read_csv("data/csv/tickers_idx.csv") #7243
to_remove = []
for i, row in tickers_idx.iterrows():
    symbol = row["ticker"]
    
    query = (select(
                db.history_table.c.Date,
                db.history_table.c.Open,
                db.history_table.c.High,
                db.history_table.c.Low,
                db.history_table.c.Close,
                db.history_table.c.Volume,
            )
            .where(db.history_table.c.Ticker == symbol)
            .order_by(db.history_table.c.Date)
            # .limit(250)
            )

    df = pd.read_sql(query, db.connection)

    df = TIME_ENCODER(df).encode()
    array = df.to_numpy()
    h5py.set_dataset(symbol+"_history", array)



5902
5902
3874
1342
5902
4461
5902
2448
5902
5424
5902
3127
4402
5902
2633
5902
5902
5902
5902
5348
5902
5902
4832
3610
5902
4002
4682
1913
1161
4797
2248
4489
1463
4828
1357
5902
1466
5902
5902
5902
1694
5902
5771
4055
1414
5513
5902
5902
4367
3118
4370
2803
1938
3107
2859
5161
5902
2033
2044
5902
5902
3674
4926
4884
5902
1231
5902
2433
5902
1109
1678
5902
1001
5902
1362
5902
3432
2236
5902
5902
5902
5902
5902
3902
5902
5902
4917
5902
5902
2493
5902
5902
4170
5902
5902
2554
5813
5384
1545
5902
5902
2206
3098
4166
1420
5902
4252
1144
5877
5067
2493
1812
5902
1301
3799
1080
1469
4821
1887
3117
2283
1357
5902
5902
1271
1413
2546
4987
2075
2596
5902
1322
5902
5902
2160
5902
1723
5710
5902
1011
5902
1315
5902
4875
5902
5902
2100
1545
5902
2328
5902
1006
2024
2379
3061
1216
5902
1316
1056
5902
2298
5902
1098
2770
5902
5631
4158
5902
5902
5902
2411
1237
1178
4173
2364
4795
5902
2015
1617
2008
1502
5097
2833
1525
1045
4590
1415
5902
5902
5902
1541
1222
5902
2688
2551
1265
2390
2791
3021
5902


In [6]:
symbol = "AAPL"
array = h5py.get_dataset(symbol+"_history")
df = pd.DataFrame(array, columns=[
    'Open', 
    'High', 
    'Low', 
    'Close',
    'Volume', 
    'year',
    'month',
    'day',
    'month_sin',
    'month_cos',
    'day_sin',
    'day_cos'])
print(df)

            Open        High         Low       Close       Volume    year  \
0       0.795873    0.853737    0.771683    0.849468  535796800.0  2000.0   
1       0.821485    0.839508    0.767889    0.777850  512377600.0  2000.0   
2       0.787335    0.839034    0.781644    0.789232  778321600.0  2000.0   
3       0.805359    0.811999    0.720933    0.720933  767972800.0  2000.0   
4       0.732317    0.766466    0.724728    0.755083  460734400.0  2000.0   
...          ...         ...         ...         ...          ...     ...   
5897  181.270004  183.889999  180.970001  183.789993   54274900.0  2023.0   
5898  182.800003  184.149994  182.440002  183.309998   54929100.0  2023.0   
5899  183.369995  184.389999  182.020004  183.949997   57462900.0  2023.0   
5900  183.960007  186.520004  183.779999  186.009995   65433200.0  2023.0   
5901  186.729996  186.990005  184.270004  184.919998  101235600.0  2023.0   

      month   day     month_sin  month_cos   day_sin   day_cos  
0       1.